In [1]:
import os 
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/MR-Khan0099/Data-Science-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "MR-Khan0099"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0178a4fbac2dd7881b940e3db90d6333039f5709"

In [2]:
%pwd

'c:\\Users\\Rizwan\\Desktop\\Courses\\E3E DS Project\\Data-Science-Project\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Rizwan\\Desktop\\Courses\\E3E DS Project\\Data-Science-Project'

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [21]:
from src.DataScienceProject.constants import *
from src.DataScienceProject.utils.common import read_yaml, create_directories, save_json


In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/MR-Khan0099/Data-Science-Project.mlflow"


        )
        return model_evaluation_config


In [23]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [41]:
import mlflow.sklearn


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            if tracking_url_type_store != "file":
                # Register the model in the Model Registry
                mlflow.sklearn.autolog(model, "model", registered_model_name="ElasticnetModel")
            else:
                # Log the model without registering it
                mlflow.sklearn.log_model(model, "model")
    


In [42]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-07 01:54:12,211: INFO: common: YAML file loaded successfully from config\config.yaml]
[2025-07-07 01:54:12,214: INFO: common: YAML file loaded successfully from params.yaml]
[2025-07-07 01:54:12,219: INFO: common: YAML file loaded successfully from schema.yaml]
[2025-07-07 01:54:12,223: INFO: common: Created directory: artifacts]
[2025-07-07 01:54:12,225: INFO: common: Created directory: artifacts/model_evaluation]
[2025-07-07 01:54:12,635: INFO: common: Data saved to JSON file at artifacts\model_evaluation\metrics.json]
🏃 View run delicate-snail-168 at: https://dagshub.com/MR-Khan0099/Data-Science-Project.mlflow/#/experiments/0/runs/d653c82d174d44cc80f1d17391473c0b
🧪 View experiment at: https://dagshub.com/MR-Khan0099/Data-Science-Project.mlflow/#/experiments/0
